In [54]:
#pip install pymupdf pdf2image opencv-python pytesseract pdfplumber
!pip install pymupdf pdf2image opencv-python pytesseract pdfplumber
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [55]:
#!/usr/bin/env python3
# -*- coding:utf-8 -*-

## System Libraries ##
import sys
import os

## Data Analysis Libraries ##
from pprint import pprint
import csv
import json
import random
from random import sample
import pandas as pd
import time
from pathlib import Path
import numpy as np

## NLP Libraries ##
import re
import string
#from gtts import gTTS
#import nltk
#from nltk import sent_tokenize
#from nltk import tokenize
#from pyzhuyin import pinyin_to_zhuyin  #, zhuyin_to_pinyin

# pdf
#import fitz  # PyMuPDF
#from pdf2image import convert_from_path
#import cv2
#import pytesseract

import fitz  # PyMuPDF for text extraction
from pdf2image import convert_from_path
import cv2
import pytesseract
from google.colab import files

In [39]:
def extract_text(pdf_path):
    """
    Function to extract text from PDF
    """
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

def extract_graphs(pdf_path, output_folder="graphs"):
    """
    Function to extract graphs (images) from PDF
    """
    images = convert_from_path(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    for i, image in enumerate(images):
        img_path = os.path.join(output_folder, f"page_{i+1}.png")
        image.save(img_path, "PNG")

        # Read image using OpenCV
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Apply edge detection
        edges = cv2.Canny(img, 50, 150)

        # Find contours (potential graphs)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for j, cnt in enumerate(contours):
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 100 and h > 100:  # Filter small contours
                graph = img[y:y+h, x:x+w]
                cv2.imwrite(os.path.join(output_folder, f"graph_{i+1}_{j+1}.png"), graph)

    print(f"Graphs saved in {output_folder}")

In [40]:
# Set Tesseract Path (Adjust if necessary)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # Update for your system

# Function to extract text from PDF
def extract_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Function to extract graphs (images) from PDF and apply OCR
def extract_graphs_with_ocr(pdf_path, output_folder="graphs"):
    images = convert_from_path(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    extracted_ocr_text = []

    for i, image in enumerate(images):
        img_path = os.path.join(output_folder, f"page_{i+1}.png")
        image.save(img_path, "PNG")

        # Read image using OpenCV
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Apply edge detection
        edges = cv2.Canny(img, 50, 150)

        # Find contours (potential graphs)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for j, cnt in enumerate(contours):
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 100 and h > 100:  # Filter small contours
                graph = img[y:y+h, x:x+w]
                graph_path = os.path.join(output_folder, f"graph_{i+1}_{j+1}.png")
                cv2.imwrite(graph_path, graph)

                # Perform OCR on the graph
                graph_text = pytesseract.image_to_string(graph, config="--psm 6")
                extracted_ocr_text.append(f"Graph {i+1}_{j+1} Text:\n{graph_text}\n")

    # Save extracted text from graphs
    ocr_text_path = os.path.join(output_folder, "ocr_extracted_text.txt")
    with open(ocr_text_path, "w", encoding="utf-8") as f:
        f.writelines(extracted_ocr_text)

    print(f"Graphs saved in {output_folder}")
    print(f"OCR extracted text saved to {ocr_text_path}")

In [41]:
# Set Tesseract Path (Adjust if necessary)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # Change this for your system

# Function to extract text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

# Function to perform OCR on PDF pages
def extract_text_with_ocr(pdf_path, output_folder="ocr_pages"):
    images = convert_from_path(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    extracted_text = []

    for i, image in enumerate(images):
        img_path = os.path.join(output_folder, f"page_{i+1}.png")
        image.save(img_path, "PNG")

        # Convert to grayscale and apply OCR
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        text = pytesseract.image_to_string(img, config="--psm 6")

        extracted_text.append(f"Page {i+1} OCR Text:\n{text}\n")

    # Save extracted OCR text
    ocr_text_path = os.path.join(output_folder, "ocr_extracted_text.txt")
    with open(ocr_text_path, "w", encoding="utf-8") as f:
        f.writelines(extracted_text)

    print(f"OCR extracted text saved to {ocr_text_path}")
    return "\n".join(extracted_text)

# Function to extract graphs (images) from PDF
def extract_graphs_with_ocr(pdf_path, output_folder="graphs"):
    images = convert_from_path(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    extracted_ocr_text = []

    for i, image in enumerate(images):
        img_path = os.path.join(output_folder, f"page_{i+1}.png")
        image.save(img_path, "PNG")

        # Read image using OpenCV
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Apply edge detection
        edges = cv2.Canny(img, 50, 150)

        # Find contours (potential graphs)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for j, cnt in enumerate(contours):
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 100 and h > 100:  # Filter small contours
                graph = img[y:y+h, x:x+w]
                graph_path = os.path.join(output_folder, f"graph_{i+1}_{j+1}.png")
                cv2.imwrite(graph_path, graph)

                # Perform OCR on the graph
                graph_text = pytesseract.image_to_string(graph, config="--psm 6")
                extracted_ocr_text.append(f"Graph {i+1}_{j+1} Text:\n{graph_text}\n")

    # Save extracted text from graphs
    ocr_text_path = os.path.join(output_folder, "ocr_graph_text.txt")
    with open(ocr_text_path, "w", encoding="utf-8") as f:
        f.writelines(extracted_ocr_text)

    print(f"Graphs saved in {output_folder}")
    print(f"OCR extracted text from graphs saved to {ocr_text_path}")


In [42]:
## Check if the pdf need OCR *(apparently all of it need OCR)
#import fitz  # PyMuPDF

def is_machine_readable(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        text = page.get_text("text")
        if text.strip():  # If there is any text, it's machine-readable
            return True
    return False

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [58]:
if __name__ == "__main__":
    # Get the pdf files in the folder
    pdf_datapath = Path("/content/drive/MyDrive/0_2024_Mine_PreProject/2025_internship_Digital Business Development/Vimmax/wetransfer_vimmax-catalogue_2025-03-05_0616")
    #pdf_fileLIST = [path.name for path in IMF_DIR.iterdir() if re.match(r'Alice_IF_IMF_*', path.name)]
    pdf_fileLIST = []
    for path in pdf_datapath.iterdir():
      if re.match(r"VIMMAX*", path.name):
        pdf_fileLIST.append(path.name)
    pprint(pdf_fileLIST)
    print(len(pdf_fileLIST))
    """
    # See if OCR is needed
    for pdf in pdf_fileLIST:
      pdf_file = pdf_datapath / Path(pdf)
      if is_machine_readable(pdf_file):
          print(str(pdf), ">> PDF is machine-readable.")
      else:
          extract_text_with_ocr(pdf_file)
          #print(str(pdf), ">> PDF contains images, OCR is required.")
    """

['VIMMAX_KITCHENWARE_MARBLE SERVING BOARD SERIES_CATALOGUE_01_V1-1.pdf',
 'VIMMAX_TABLEWARE COLLECTION_CATALOGUE_02_V1-1.pdf',
 'VIMMAX_KITCHENWARE_KITCHEN KNIVES_02_V1-1.pdf',
 'VIMMAX_ TABLEWARE_DOUBLE WALL STAINLESS STEEL SERIES_CATALOGUE_01_V1-3.pdf',
 'VIMMAX_CARAFE_GLASS SERIES_CATALOGUE_01_ V1-1.pdf',
 'VIMMAX_TABLEWARE_MODERN SERIES_CATALOGUE_01_ V1-1.pdf',
 'VIMMAX_HOME & OFFICE DECORATIONS_DECOR CERAMIC SERIES_CATALOGUE_01_V1-1.pdf',
 'VIMMAX_KITCHENWARE_WOODEN SERVING BOARD SERIES_CATALOGUE_01_V1-1.pdf',
 'VIMMAX_BUFFETWARE_SERVING BOWL_TRAY_WITH STAND_GLASS '
 'SERIES_CATALOGUE_01_V1-1.pdf',
 'VIMMAX_SERVINGWARE_FROSTED STAINLESS STEEL SERIES_CATALOGUE_01_V1-1.pdf',
 'VIMMAX_SERVINGWARE_SNOWFLAKE STAINLESS STEEL SERIES_CATALOGUE_01_V2-1.pdf',
 'VIMMAX_BUFFETWARE_METAL RISERS SERIES_CATALOGUE_01_V2-1.pdf',
 'VIMMAX_SP MILL_CERAMIC GRINDER_CATALOGUE_01_V1-1.pdf',
 'VIMMAX_HIGH SPEED OVEN ACCESSORIES_PADDLE_NONSTICK & MESH '
 'BASKET_CATALOGUE_01_ V1-1.pdf',
 'VIMMAX_PROFESSIO

In [66]:
# Get the name and the type and tags along the way

if __name__ == "__main__":
  # Get the pdf files in the folder
  pdf_datapath = Path("/content/drive/MyDrive/0_2024_Mine_PreProject/2025_internship_Digital Business Development/Vimmax/wetransfer_vimmax-catalogue_2025-03-05_0616")
  #pdf_fileLIST = [path.name for path in IMF_DIR.iterdir() if re.match(r'Alice_IF_IMF_*', path.name)]
  pdf_fileLIST = []
  for path in pdf_datapath.iterdir():
    if re.match(r"VIMMAX*", path.name):
      pdf_fileLIST.append(path.name.lower())
  pprint(pdf_fileLIST)
  print(len(pdf_fileLIST))

  typeLIST = []
  subtypeLIST = []
  for pdfSTR in pdf_fileLIST:
    nameLIST = pdfSTR.split("_")
    print(nameLIST)

    typeLIST.append(nameLIST[1])
    subtypeLIST.append(nameLIST[2])

  pprint(typeLIST)
  pprint(subtypeLIST)


['vimmax_bakeware_pizza tools series_catalogue_01_v1-3.pdf',
 'vimmax_buffetware_serving bowl_tray_with stand_glass '
 'series_catalogue_01_v1-1.pdf',
 'vimmax_kitchenware_wooden serving board series_catalogue_01_v1-1.pdf',
 'vimmax_kitchenware_marble serving board series_catalogue_01_v1-1.pdf',
 'vimmax_home & office decorations_decor ceramic series_catalogue_01_v1-1.pdf',
 'vimmax_buffetware_glass serving plates with decals '
 'series_catalogue_01_v1-2.pdf',
 'vimmax_kitchenware_kitchen knives_02_v1-1.pdf',
 'vimmax_kitchen tools & gadgets_glass_stainless steel_sp shakers '
 'series_02_v1-1.pdf',
 'vimmax_high speed oven accessories_paddle_nonstick & mesh '
 'basket_catalogue_01_ v1-1.pdf',
 'vimmax_drinkware_pet tumbler series_catalogue_02_v1-1.pdf',
 'vimmax_carafe_glass series_catalogue_01_ v1-1.pdf',
 'vimmax_buffetware_metal risers series_catalogue_01_v2-1.pdf',
 'vimmax_sp & ov_cruet set_amber glass series_catalogue_01_ v2-2.pdf',
 'vimmax_combi oven accessories_catalogue_01_ v

In [51]:
# Colab-friendly version of OCR extraction
# Set Tesseract Path (Colab already has Tesseract installed)
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

"""
# Upload a PDF file
uploaded = files.upload()

# Get the first uploaded file
pdf_path = list(uploaded.keys())[0]
"""

# Create output folders
os.makedirs("extracted_graphs", exist_ok=True)
os.makedirs("ocr_output", exist_ok=True)

# Function to extract text from PDF
def extract_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"

    # Save text to a file
    with open("ocr_output/extracted_text.txt", "w", encoding="utf-8") as f:
        f.write(text)

    print("\nExtracted text saved to: ocr_output/extracted_text.txt")
    return text

# Function to extract graphs and apply OCR
def extract_graphs_with_ocr(pdf_path):
    images = convert_from_path(pdf_path)
    extracted_ocr_text = []

    for i, image in enumerate(images):
        img_path = f"extracted_graphs/page_{i+1}.png"
        image.save(img_path, "PNG")

        # Read image using OpenCV
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Apply edge detection
        edges = cv2.Canny(img, 50, 150)

        # Find contours (potential graphs)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for j, cnt in enumerate(contours):
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 100 and h > 100:  # Filter small contours
                graph = img[y:y+h, x:x+w]
                graph_path = f"extracted_graphs/graph_{i+1}_{j+1}.png"
                cv2.imwrite(graph_path, graph)

                # Perform OCR on the graph
                graph_text = pytesseract.image_to_string(graph, config="--psm 6")
                extracted_ocr_text.append(f"Graph {i+1}_{j+1} Text:\n{graph_text}\n")

    # Save extracted OCR text from graphs
    with open("ocr_output/ocr_graph_text.txt", "w", encoding="utf-8") as f:
        f.writelines(extracted_ocr_text)

    print("\nGraphs saved in the 'extracted_graphs' folder")
    print("OCR extracted text from graphs saved to: ocr_output/ocr_graph_text.txt")

In [53]:
if __name__ == "__main__":
    # Get the pdf files in the folder
    pdf_datapath = Path("/content/drive/MyDrive/0_2024_Mine_PreProject/2025_internship_Digital Business Development/Vimmax/wetransfer_vimmax-catalogue_2025-03-05_0616")
    #pdf_fileLIST = [path.name for path in IMF_DIR.iterdir() if re.match(r'Alice_IF_IMF_*', path.name)]
    pdf_fileLIST = []
    for path in pdf_datapath.iterdir():
      if re.match(r"VIMMAX*", path.name):
        pdf_fileLIST.append(path.name)
    #pprint(pdf_fileLIST)

    # Apply OCR onto every file
    for pdf in pdf_fileLIST[:2]:
      pdf_file = pdf_datapath / Path(pdf)
      # Run text and graph extraction
      extracted_text = extract_text(pdf_file)
      #extract_graphs_with_ocr(pdf_file)

      # Zip the results for easy download
      #!zip -r extracted_graphs.zip extracted_graphs
      !zip -r ocr_output.zip ocr_output

      print("\nDownload your extracted files:")
      #print("➡️ Click to download extracted graphs: extracted_graphs.zip")
      print("➡️ Click to download extracted text & OCR results: ocr_output.zip")


Extracted text saved to: ocr_output/extracted_text.txt
  adding: ocr_output/ (stored 0%)
  adding: ocr_output/extracted_text.txt (stored 0%)

Download your extracted files:
➡️ Click to download extracted text & OCR results: ocr_output.zip

Extracted text saved to: ocr_output/extracted_text.txt
updating: ocr_output/ (stored 0%)
updating: ocr_output/extracted_text.txt (deflated 87%)

Download your extracted files:
➡️ Click to download extracted text & OCR results: ocr_output.zip


In [ ]:
    images = convert_from_path(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    extracted_text = []

    for i, image in enumerate(images):
        img_path = os.path.join(output_folder, f"page_{i+1}.png")
        image.save(img_path, "PNG")

        # Convert to grayscale and apply OCR
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        text = pytesseract.image_to_string(img, config="--psm 6")

        extracted_text.append(f"Page {i+1} OCR Text:\n{text}\n")

    # Save extracted OCR text
    ocr_text_path = os.path.join(output_folder, "ocr_extracted_text.txt")
    with open(ocr_text_path, "w", encoding="utf-8") as f:
        f.writelines(extracted_text)

    print(f"OCR extracted text saved to {ocr_text_path}")
    return "\n".join(extracted_text)